## Imports

### Imports & Stuff

In [1]:
import pandas as pd
## pandas 1.1.1 necessary
import numpy as np
import string 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import math
from gensim.models import Word2Vec
from plotly.graph_objects import Scatter3d

In [35]:
#Getting the merger data
merger = pd.read_pickle("../../raw_data/merger")

In [36]:
#Getting all the data
data = pd.read_pickle("../../delphes/data/update_age_sex")
data = data.reset_index()

## Cleaning

In [37]:
#reducing the dataset
extract = merger[["index","content_y"]]

### Dropping the None

In [122]:
#identifying the None
count = 0
dropables = []
for i in extract["content_y"]:
    if i == None:
        dropables.append(count)
        count+= 1
    else:
        count+=1

In [39]:
#removing the Nones
extract = extract.drop(labels=dropables)

In [43]:
extract = extract.reset_index()

### Creating a Dictionary with Averaged Arrays

In [44]:
#Creating a dict_tweets with Index and Averaged Array
tweets = []
check = []
dict_tweets = {}
for i in extract["level_0"]:
    result = 0
    prov_results = []
    if len(extract["content_y"][i]) > 1: 
        for j in range (0,len(extract["content_y"][i])): 
            if extract["content_y"][i] != False: 
                prov_results.append(extract["content_y"][i][j])
            else:
                pass
        dict_tweets[extract["index"][i]] = (sum(prov_results)/len(extract["content_y"][i]))
        check.append(i)
    else: 
        pass
        check.append(i)

KeyError: 11183

### Deleting Floats and Other Falses

In [46]:
### Removing the Falses within the Sentences
floats = []
for i,j in dict_tweets.items(): 
    if type(j) != np.ndarray:
        floats.append(i)

In [47]:
for i in floats:
    del dict_tweets[i]

### Creating DF

In [48]:
d_i_tweets = pd.DataFrame(dict_tweets).transpose()

In [49]:
d_i_tweets = d_i_tweets.reset_index()

In [51]:
d_i_tweets

,index,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,5076,-0.016846,-0.011860,-0.042851,0.044163,-0.027453,0.022471,0.000556,-0.036933,-0.001978,...,0.010223,-0.011244,0.009240,0.028963,-0.008443,-0.042849,-0.014880,0.041051,0.023303,-0.012236
1,5079,-0.025288,-0.020077,-0.029960,0.033417,-0.022732,0.031891,0.014065,-0.054808,0.008609,...,0.014924,-0.009034,0.017069,0.036944,0.000622,-0.024272,-0.017961,0.043717,0.026246,-0.006339
2,5080,-0.026789,0.006986,-0.025454,0.034253,-0.019140,0.003891,-0.009987,-0.062290,-0.005792,...,0.033633,-0.010781,0.009733,-0.002281,0.006047,-0.046107,-0.023379,0.041657,0.028609,-0.009291
3,5081,-0.018487,-0.005300,-0.019736,0.044298,-0.020414,0.032369,0.003270,-0.053760,-0.012529,...,0.008235,-0.011501,0.014615,0.038758,0.009990,-0.015177,-0.028269,0.005696,0.034646,0.007942
4,5082,-0.010963,-0.039525,-0.020190,0.044142,-0.027780,0.032248,0.009840,-0.049892,0.001871,...,0.018865,-0.011564,0.013905,0.023749,0.010506,-0.017543,-0.027306,0.027216,0.015570,0.010784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,133994,-0.029210,0.011155,-0.006377,0.046776,-0.025457,0.002079,0.006012,-0.028870,0.015737,...,0.037759,-0.015325,0.009431,-0.009702,0.009091,-0.043326,-0.025808,0.038204,0.031567,-0.006687
10688,133995,-0.041173,0.010350,-0.021792,0.045523,-0.020995,0.017644,0.002863,-0.055886,-0.019077,...,0.017315,0.016870,0.006894,-0.020515,0.003003,-0.046619,-0.022752,0.059461,0.049146,-0.007352
10689,133997,-0.030016,0.003368,-0.033410,0.023910,-0.019755,0.018780,0.005391,-0.040908,-0.017813,...,0.022029,-0.017187,0.017818,-0.004375,-0.000672,-0.035493,-0.011981,0.056631,0.027491,-0.012153
10690,133998,-0.006542,0.005297,-0.014884,0.020711,-0.029233,0.003898,-0.024561,-0.053010,-0.021391,...,0.029530,0.007000,-0.000413,-0.044168,0.037959,-0.042401,-0.030223,0.052912,0.042841,-0.019456


### Merging DFs

In [52]:
all_info = data.merge(d_i_tweets,on="index")

## Label Encoding

### Country

In [53]:
def countries(values):
    if values == "Netherlands":
        return 0
    elif values == "Denmark":
        return 1
    else:
        return 2

In [57]:
all_info["countries"] = all_info["country"].apply(countries)

### European Group

In [58]:
def groups(values):
    if values == 'Group of the Greens/European Free Alliance':
        return 0
    elif values == 'Renew Europe Group':
        return 1
    elif values == "Group of the European People's Party (Christian Democrats)":
        return 2
    elif values == 'Group of the Progressive Alliance of Socialists and Democrats in the European Parliament':
        return 3
    elif values == 'European Conservatives and Reformists Group':
        return 4
    elif values == 'Group of the European United Left - Nordic Green Left':
        return 5
    elif values == 'Non-attached Members':
        return 6
    else:
        return 7

In [59]:
all_info["european_group"] = all_info["group"].apply(groups)

### National Party

In [60]:
parties_list = ['Socialistisk Folkeparti',
       'Volkspartij voor Vrijheid en Democratie',
       'Christen Democratisch Appèl', 'Partij van de Arbeid',
       'ChristenUnie', 'GroenLinks', 'Forum voor Democratie',
       'Socialdemokratiet', 'Partij voor de Dieren', 'Democraten 66',
       'Venstre, Danmarks Liberale Parti', 'Det Radikale Venstre',
       'GO Realisme & Daadkracht', 'Staatkundig Gereformeerde Partij',
       'Enhedslisten', 'Det Konservative Folkeparti']

def parties(values):
    for i in parties_list:
        if values == i:
            return parties_list.index(i)

In [70]:
all_info["nat_party_code"] = all_info["nat_group"].apply(parties)

### Age

In [84]:
def age_func(values):
    if values <= 30:
        return 0
    if (values > 30) & (values <= 40):
        return 1
    if (values > 40) & (values <=50):
        return 2
    if (values > 50) & (values <= 60):
        return 3
    if values > 60:
        return 4

In [86]:
all_info["age_code"].unique()

array([0, 3, 2, 4, 1])

In [85]:
all_info["age_code"] = all_info["age"].apply(age_func)

## Exporting Document

### Exporting Words Matrix

In [63]:
d_i_tweets.to_csv('d_i_tweets.tsv',sep="\t", index=False, header=False)

### Exporting Labels

In [87]:
all_info[["countries","european_group","nat_party","sex","age_code"]].to_csv("multi_variables.tsv",sep="\t", index=False, header=True)

## Merger Attempts



In [119]:
port_irl = pd.read_pickle("../../raw_data/merger2")
port_irl = port_irl.reset_index()
port_irl = port_irl[["index","content"]]

In [120]:
data.merge()

,index,content
0,20738,"[False, False, [-0.0564866, 0.0520961, -0.0289..."
1,20739,"[False, [0.0680034, -0.0906367, -0.0319666, 0...."
2,20740,"[False, False, False, False, False, False, Fal..."
3,20741,"[[-0.0393396, 0.0268131, -0.142318, 0.0613933,..."
4,20742,"[[-0.0393396, 0.0268131, -0.142318, 0.0613933,..."
...,...,...
7795,130490,"[[0.0109718, -0.0593663, -0.0322032, 0.0416514..."
7796,130491,"[[0.0186351, -0.0140546, -0.0420493, -0.067852..."
7797,130492,"[[0.0489321, -0.0598671, -0.0264063, -0.014537..."
7798,130493,"[[-0.133483, 0.0550787, -0.0270976, 0.00313864..."
